In [1]:
import sys
print("python 版本检测",sys.version)

python 版本检测 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]


In [2]:
#gpu检测
import torch
print(f"PyTorch version: {torch.__version__}")
!nvidia-smi
print("torch可使用的GPU数量是",torch.cuda.device_count())

PyTorch version: 2.0.0+cu117
Thu Jun 13 09:07:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:09.0 Off |                    0 |
| N/A   49C    P0    30W /  70W |    967MiB / 15360MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                           

In [3]:
# %pip install psutil gputil ipywidgets IPython

In [4]:
import psutil
import GPUtil
import ipywidgets as widgets
from IPython.display import display
import time
import threading

# 监控线程的终止标志
stop_thread = threading.Event()

cpu_label = widgets.Label()
gpu_label = widgets.Label()

def update_metrics():
    while not stop_thread.is_set():
        # Update CPU usage
        cpu_usage = psutil.cpu_percent(interval=1)
        cpu_label.value = f"CPU 使用率: {cpu_usage}%"
        
        # Update GPU usage
        gpus = GPUtil.getGPUs()
        gpu_info = ""
        for gpu in gpus:
            gpu_info += (f"GPU {gpu.id}: GPU使用率: {gpu.load * 100}%, "
                         f"显存总量: {gpu.memoryTotal}MB, "
                         f"已用显存: {gpu.memoryUsed}MB, "
                         f"可用显存: {gpu.memoryFree}MB\n")
        gpu_label.value = gpu_info
        
        # 使用较短的 sleep 间隔来检查终止标志
        for _ in range(10):
            if stop_thread.is_set():
                break
            time.sleep(0.1)

# 启动监控线程
thread = threading.Thread(target=update_metrics)
thread.start()
display(cpu_label)
display(gpu_label)
stop_button = widgets.Button(description="停止监控")
display(stop_button)
def stop_monitoring(b):
    stop_thread.set()
    thread.join()  # 等待线程结束
    print("监控已停止")

stop_button.on_click(stop_monitoring)

Label(value='')

Label(value='')

Button(description='停止监控', style=ButtonStyle())

In [6]:
import jittor as jt
from jittor import nn

class LinearModel(nn.Module):
    def __init__(self):
        self.linear = nn.Linear(10, 2)
    def execute(self, x):
        x = self.linear(x)
        return x
net = LinearModel().cuda()
x = jt.random((10, 10))
out = net(x)
print(out.shape)
net


[i 0613 09:08:23.497532 76 log.cc:351] Load log_sync: 1
[i 0613 09:08:23.633205 76 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/envs/pyt/lib/python3.10/site-packages/jittor
[i 0613 09:08:23.635853 76 compiler.py:957] g++ at /usr/bin/g++(9.4.0)
[i 0613 09:08:23.636378 76 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xf1/IntelRXeonRPlax40/default
[i 0613 09:08:23.639747 76 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.7.99) at /usr/local/cuda/bin/nvcc.
[i 0613 09:08:23.642619 76 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0613 09:08:23.744465 76 compiler.py:1011] cuda key:cu11.7.99_sm_75
[i 0613 09:08:24.091705 76 __init__.py:227] Total mem: 30.60GB, using 10 procs for compiling.
[i 0613 09:08:24.331670 76 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0613 09:08:26.336247 76 init.cc:62] Found cuda archs: [75,]
[i 0613 09:08:27.511953 76 cuda_flags.cc:49] CUDA enabled.

Compiling Operators(3/3) used: 2.31s

[10,2,]


LinearModel(
    linear: Linear(10, 2, float32[2,], None)
)

In [19]:
net = LinearModel().cuda(0)
jt.flags.use_cuda = 1

In [22]:
net.cuda(device='gpu')

LinearModel(
    linear: Linear(10, 2, float32[2,], None)
)